# LangChain 入门示例

本 notebook 将带你了解 langchain 的一些基础功能。我们会演示如何：

1. 安装所需的依赖包
2. 导入 langchain 库
3. 构建一个简单的 LLM Chain 来实现英法翻译

注意：你需要拥有 OpenAI 的 API key，并将其设置在环境变量 `OPENAI_API_KEY` 中，或者在代码中直接配置。

In [12]:
# 安装必要的依赖包
!pip install langchain openai
!pip install langchain-community
!pip install google-generativeai langchain-google-genai


  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.7 MB/s eta 0:00:00a 0:00:01
Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 3.0 MB/s eta 0:00:0000:0100:01
Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl (9.3

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

dotenv_path = os.path.join('..', '.env')
load_dotenv(dotenv_path)

google_api_key = os.environ.get('GOOGLE_API_KEY')
print(f"Google API Key available: {'Yes' if google_api_key else 'No'}")

# 如果没有在环境变量中找到，您可以直接设置
if not google_api_key:
    # 这里不要直接写入真实密钥，仅作示例
    # google_api_key = "your-google-api-key-here" 
    print("没有找到 Google API 密钥，请确保在 .env 文件中设置 GOOGLE_API_KEY")

# 创建一个翻译的 Prompt 模板
prompt_template = "Translate the following English text to Chinese, straightly return the result: {text}"
prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

# 初始化 Google AI 模型
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=google_api_key,
    temperature=0
)

# 利用 PromptTemplate 和 LLM 构建 LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# 运行链任务：将英文翻译为中文
try:
    result = llm_chain.run(text="Hello, how are you?")
    print("翻译结果：", result)
except Exception as e:
    print(f"运行时出错: {str(e)}")

Google API Key available: Yes
翻译结果： 你好，你好吗？


但是为什么叫“LLM Chain”呢🤔？
接下来，构建一个（比较）复杂的 LLM Chain：

用户提出一个问题，系统：

使用维基百科 API 搜索相关文章。
提取文章的摘要。
将问题和摘要组合成一个提示。
将提示发送给 LLM。
返回 LLM 生成的答案。

In [21]:
!pip install wikipedia

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11727 sha256=411bfd951f0295774ac92051102ce6fabeff139253ee5d062c27dbbf763609b4
  Stored in directory: /Users/a51/Library/Caches/pip/wheels/c2/46/f4/caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia


In [22]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.utilities import WikipediaAPIWrapper
from dotenv import load_dotenv

dotenv_path = os.path.join('..', '.env')
load_dotenv(dotenv_path)

google_api_key = os.environ.get('GOOGLE_API_KEY')
print(f"Google API Key available: {'Yes' if google_api_key else 'No'}")

Google API Key available: Yes


In [23]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=google_api_key,
    temperature=0.3  # 稍微增加一点随机性
)

wikipedia = WikipediaAPIWrapper()

prompt_template = """
   基于提供的上下文回答用户的问题：
    Question: {question}
    Context: {context}
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

context = wikipedia.run(question)

这行代码是在使用LangChain的Wikipedia工具来进行信息检索。具体来说：
wikipedia 是一个LangChain的工具或检索器，通常是通过 WikipediaAPIWrapper 类创建的实例
run(question) 方法接收一个问题或查询字符串作为输入
该方法会在维基百科上搜索与问题相关的文章内容
返回的结果 context 包含了从维基百科获取的相关信息，这些信息通常用作大语言模型的上下文
这种模式在构建RAG（检索增强生成）应用时很常见，先检索相关信息，然后将这些信息提供给语言模型，使其生成更准确、更有依据的回答。

In [24]:
print(wikipedia.run("C, XOXO"))

Page: C,XOXO
Summary: C,XOXO is the fourth studio album by American singer Camila Cabello. It was released on June 28, 2024, through Geffen and Interscope Records. It is her first album following her departure from Epic Records.
The album was preceded by the release of the singles "I Luv It" with Playboi Carti, which peaked at number 81 on the Billboard Hot 100 and "He Knows" with Lil Nas X, as well as the promotional single "Chanel No. 5". Following the album's release, "Hot Uptown" featuring Drake was released as the third single, and "Godspeed" was released as the fourth single on September 6, 2024, alongside the Magic City Edition, which features 4 additional tracks. Guest vocalists include Playboi Carti, Lil Nas X, JT and Yung Miami of City Girls, BLP Kosher, Drake, and uncredited vocals from PinkPantheress, with the deluxe including a collaboration with Eem Triplin. The album was met with mixed-to-positive reviews by the music critics.



Page: Yours, C Tour
Summary: The Yours, C

WikipediaAPIWrapper 返回的 Summary 不是由 LLM 总结的，而是直接从维基百科 API 获取的维基百科页面摘要。

In [27]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

def ask_question(question):
    """
    使用维基百科检索信息并使用 LLM 回答问题。
    """
    try:
        # 5.1 使用维基百科搜索相关文章
        context = wikipedia.run(question)
        print(context)

        # 5.2 运行 LLMChain
        answer = llm_chain.run(context=context, question=question)

        return answer
    except Exception as e:
        return f"出现错误: {e}"

print(ask_question("C, XOXO 是哪位歌手的第几张专辑？"))

No good Wikipedia Search Result was found
由于我无法在维基百科上找到相关信息，我无法回答你关于 C, XOXO 是哪位歌手的第几张专辑的问题。


果然，因为自然语言处理匹配的关系是，失败了！接下来我们需要通过 LLM 重新构建搜索的关键词，再进行回答🤔。

In [31]:
rewrite_prompt_template = """
   请将以下中文问题改写为简洁、准确的英文关键词或短语，以便在维基百科上进行搜索。注意，是在维基百科搜索（而不是谷歌/Reddit），所以一般来说只需要搜索问题中的专有名词即可。请直接返回搜索关键词。
   Question: {question}
"""

rewrite_prompt = PromptTemplate(
    template=rewrite_prompt_template,
    input_variables=["question"]
)

rewrite_chain = LLMChain(llm=llm, prompt=rewrite_prompt)

#重新定义回答函数
def ask_question(question):
    """
    使用维基百科检索信息并使用 LLM 回答问题。
    """
    try:
        # 使用 LLM 重写查询字符串
        search_query = rewrite_chain.run(question)
        print(f"重写后的查询字符串: {search_query}")

        # 使用维基百科搜索相关文章
        context = wikipedia.run(search_query)
        print(f"维基百科摘要: {context}")

        # 运行 LLMChain
        answer = llm_chain.run(context=context, question=question)

        return answer
    except Exception as e:
        return f"出现错误: {e}"
    
print(ask_question("C, XOXO 是哪位歌手的第几张专辑？"))

重写后的查询字符串: C, XOXO
维基百科摘要: Page: C,XOXO
Summary: C,XOXO is the fourth studio album by American singer Camila Cabello. It was released on June 28, 2024, through Geffen and Interscope Records. It is her first album following her departure from Epic Records.
The album was preceded by the release of the singles "I Luv It" with Playboi Carti, which peaked at number 81 on the Billboard Hot 100 and "He Knows" with Lil Nas X, as well as the promotional single "Chanel No. 5". Following the album's release, "Hot Uptown" featuring Drake was released as the third single, and "Godspeed" was released as the fourth single on September 6, 2024, alongside the Magic City Edition, which features 4 additional tracks. Guest vocalists include Playboi Carti, Lil Nas X, JT and Yung Miami of City Girls, BLP Kosher, Drake, and uncredited vocals from PinkPantheress, with the deluxe including a collaboration with Eem Triplin. The album was met with mixed-to-positive reviews by the music critics.



Page: Yours, C